this notebook is used to visiualize and analize shock regions

In [ ]:
from postprocessing import *
import glob
import os
import healpy as hp
from natsort import natsorted

In [ ]:
simulation_directory = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100/jet44_early_shockfinder')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
filename = "shocks_%03d.hdf5" % (1)
shock_data = h5py.File(output_directory + filename, "r")
shock = shock_data['PartType0/Machnumber'][:]

In [ ]:
shock[shock > 0].max()

In [ ]:
shock[distance.argsort()[:3]].mean()

In [ ]:
for i_file in range(6):

    filename = "shocks_%03d.hdf5" % (i_file)
    shock_data = h5py.File(output_directory + filename, "r")
    shock = shock_data['PartType0/Machnumber'][:]
    bins = np.linspace(0, 150, 35)

    fig, ax = plt.subplots(2, 3, figsize=(11, 6.1))
    fig.tight_layout(w_pad=7.0, h_pad=3.0)
    plot_dens_vel(ax[0][0], output_directory + "snap_%03d.hdf5" % (i_file))
    plot_temp_vel(ax[0][1], output_directory + "snap_%03d.hdf5" % (i_file))
    plot_pressure(ax[0][2], output_directory + "snap_%03d.hdf5" % (i_file))
    plot_jet_tracer(ax[1][0], output_directory + "snap_%03d.hdf5" % (i_file))
    plot_shocks(ax[1][1], output_directory + "shocks_%03d.hdf5" % (i_file))


    ax[1][2].hist(shock[shock != 0], bins=bins, log=True)
    ax[1][2].set_xlabel('shock value distribution')
    ax[1][2].set_xlim(0, 150)
    ax[1][2].set_ylim(1, 1500)
    plt.savefig(figures_directory + f'6panels_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/6panels*.png'
ofilename = figures_directory + '/6panels-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)

## mollview

In [ ]:
def find_shocks(directory, i_file):
    filename = "shocks_%03d.hdf5" % (i_file)
    snap_data = h5py.File(directory + filename, "r")
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    shocks = snap_data['PartType0/Machnumber'][:].T
    return shocks, x - 500, y - 500, z - 500, get_time_from_snap(snap_data) * unit_time_in_megayr

In [ ]:
i_file = 1
shock_all, x_all, y_all, z_all, time = find_shocks(output_directory, i_file)

In [ ]:
radius_range = np.linspace(40, 500, 31)
NSIDE = 31
NPIX = hp.nside2npix(NSIDE)
theta, phi = hp.pix2ang(nside=NSIDE, ipix=np.arange(NPIX)) # return colatitude and longtitude in radian
vec = hp.ang2vec(theta,phi)  # return unit 3D position vector

In [ ]:
for radius in radius_range:
    print(radius)
    vec_scaled = vec * radius
    mask = ((np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) < radius + 20) & 
            (np.sqrt(x_all ** 2 + y_all ** 2 + z_all ** 2) > radius - 20))
    shock, x, y, z = shock_all[mask], x_all[mask], y_all[mask], z_all[mask]
    shock_shell = []

    for vector in vec_scaled:
        distance = (vector[0] - x) ** 2 + (vector[1] - y) ** 2 + (vector[2] - z) ** 2
        shock_shell.append(shock[distance.argsort()[:3]].mean())
    


    hp.mollview(np.array(shock_shell), title=fr"Shocks in $r \in$ [{np.round(radius,0) - 20}; {np.round(radius,0) + 20}] pc", 
            min=1e-2, max=100, rot=(90,0,0), unit="mass flux in cgs", norm='log', cmap='magma')
    hp.graticule()
    plt.savefig(figures_directory + f'shocks_mollview_{i_file}_{np.round(radius, 0)}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/shocks_mollview_{i_file}_*.png'
ofilename = figures_directory + f'/shocks_mollview_{i_file}-jet.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=160, loop=0)